# Imports

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import pickle
import os
from time import time
import numpy as np

from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, GlobalAveragePooling1D, MaxPooling1D, Concatenate
from tensorflow.keras.layers import Dense, Conv1D, Dropout, BatchNormalization, LeakyReLU, ELU
from tensorflow.keras import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Softmax

# Data loading

In [3]:
n_tips = ['674', '489', '87']

In [4]:
pickle_base = '/workspace/deep_birth_death/deep_learning/pickles/simulations/dataset_'
data = dict()
for i in n_tips:
    with open(pickle_base + i + "_10k.pkl", 'rb') as f:
        data[i] = pickle.load(f)

# Model definition

In [5]:
def create_nn(n_out, n_tips, clas=False, div_scenario = None):
    input_data = Input(shape=(n_tips, 1))

    final_filters = 128
    x = Conv1D(16, kernel_size=3, padding='same')(input_data)
    x = ELU()(x)
    x = Conv1D(16, kernel_size=3, padding='same')(x)
    x = ELU()(x)
    x = MaxPooling1D(pool_size=2)(x)
    
    if n_tips > 256:
        final_filters = 64
        x = Conv1D(32, kernel_size=3, padding='same')(x)
        x = ELU()(x)
        x = Conv1D(32, kernel_size=3, padding='same')(x)
        x = ELU()(x)
        x = MaxPooling1D(pool_size=2)(x)
    
        if n_tips > 512:
            final_filters = 128
            x = Conv1D(64, kernel_size=3, padding='same')(x)
            x = ELU()(x)
            x = Conv1D(64, kernel_size=3, padding='same')(x)
            x = ELU()(x)
            x = MaxPooling1D(pool_size=2)(x)

    x = Conv1D(final_filters, kernel_size=3, padding='same')(x)
    x = ELU()(x)
    x = Dropout(0.3)(x)

    x = GlobalAveragePooling1D()(x)

    x = Dense(32)(x)
    x = ELU()(x)
    x = Dropout(0.3)(x)
    
    x = Dense(n_out, name='logits')(x)
    if clas:
        
        output_class = Softmax()(x)
        
    else:
        if div_scenario != "SAT":
            out_list = []

            for i in range(n_out):
                y = Dense(32)(x)
                y = ELU()(y)
                y = Dropout(0.3)(y)

                y = Dense(1)(y)
                y = LeakyReLU(alpha=10)(y)
                out_list.append(y)

            output_class = Concatenate()(out_list)
            
        elif div_scenario == "WW":
            out_list = []

            for i in range(n_out):
                y = Dense(32)(x)
                y = ELU()(y)
                y = Dropout(0.3)(y)

                y = Dense(1)(y)
                y = Linear(y)
                out_list.append(y)

            output_class = Concatenate()(out_list)                
            
        else:
            x = Dense(32)(x)
            x = ELU()(x)
            x = Dropout(0.3)(x)
            x = Dense(1)(x)
            output_class = LeakyReLU(alpha=10)(x)
            
    return Model(input_data, output_class)

# Training

In [6]:
callback = EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)
out_folder_path = "/workspace/deep_birth_death/deep_learning/models/"
os.makedirs(out_folder_path + 'class/', exist_ok=True)
os.makedirs(out_folder_path + 'reg/', exist_ok=True)

## Classification training

In [19]:
for i in n_tips:
    print("\nClasification training", i, 'tips')

    nn_model = create_nn(len(data[i]['y_class_train'][0]),
                         int(i), clas=True)
    nn_model.compile(loss="categorical_crossentropy",
                     optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

    start = time()
    history = nn_model.fit(data[i]['X_train'], data[i]['y_class_train'],
                           batch_size=128, epochs=1000, validation_split=0.1,
                           callbacks=[callback], verbose=0)
    elapsed_time = time()-start
    print('Elapsed time', elapsed_time)

    save_path = out_folder_path + 'class/' + i + "_classification_temperature_"

    nn_model.save(save_path + "model.keras")
    with open(save_path + "history.pkl", 'wb') as f:
            pickle.dump(history.history, f)
    with open(save_path + "model_data.pkl", 'wb') as f:
            pickle.dump([nn_model.count_params(), elapsed_time], f)


Clasification training 674 tips
Elapsed time 923.6486873626709

Clasification training 489 tips
Elapsed time 1349.151597738266

Clasification training 87 tips
Elapsed time 920.2880265712738


## Regression training

In [7]:
for i in n_tips:
    for label in np.unique(data[i]['div_info_train']):
        div_scenario = label.split('/')[1].split('_')[0]
        #for data_norm in ['_norm']:

        # Get regression values of the corresponding scenario
        X_train = data[i]['X_train'][data[i]['div_info_train'] == label]
        y_reg_train = data[i]['y_reg_norm_train'][data[i]['div_info_train'] == label]
        y_reg_train = [np.array(elem) for elem in y_reg_train]
        
        #Train the model 
        print("\nRegression training", i, 'tips,', div_scenario, 'scenario')
        nn_model = create_nn(len(y_reg_train[0]),
                             int(i), div_scenario=div_scenario)
        nn_model.compile(loss="mae", optimizer=Adam(learning_rate=0.001),
                         metrics=['mse'])
        start = time()
        history = nn_model.fit(np.expand_dims(X_train, axis=2),
                               np.expand_dims(y_reg_train, axis=2),
                               batch_size=128, epochs=1000, validation_split=0.1,
                               callbacks=[callback], verbose=0)
        elapsed_time = time()-start
        print('Elapsed time', elapsed_time)
        
        #Save the model and the model data
        save_path = out_folder_path + 'reg/' + div_scenario + '/'
        os.makedirs(save_path, exist_ok=True)
        save_path +=  i + "_regression" + '_'
        nn_model.save(save_path + "model.keras")
        with open(save_path + "history.pkl", 'wb') as f:
                pickle.dump(history.history, f)
        with open(save_path + "model_data.pkl", 'wb') as f:
                pickle.dump([nn_model.count_params(), elapsed_time], f)


Regression training 674 tips, BD scenario
Elapsed time 87.91219878196716

Regression training 674 tips, HE scenario
Elapsed time 78.98861718177795

Regression training 674 tips, ME scenario
Elapsed time 91.36036682128906

Regression training 674 tips, SAT scenario
Elapsed time 95.16788649559021

Regression training 674 tips, SR scenario
Elapsed time 110.06941652297974

Regression training 674 tips, WW scenario
Elapsed time 104.75575184822083

Regression training 489 tips, BD scenario
Elapsed time 149.27558660507202

Regression training 489 tips, HE scenario
Elapsed time 100.52225613594055

Regression training 489 tips, ME scenario
Elapsed time 139.49027013778687

Regression training 489 tips, SAT scenario
Elapsed time 94.76165795326233

Regression training 489 tips, SR scenario
Elapsed time 168.76939392089844

Regression training 489 tips, WW scenario
Elapsed time 175.36655187606812

Regression training 87 tips, BD scenario
Elapsed time 73.76034808158875

Regression training 87 tips, 